<a href="https://colab.research.google.com/github/salman-aziz-4425/Pedestrian-Classification/blob/main/CV_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import cv2
import os
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.decomposition import PCA
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import torch
import torchvision.models as models
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
from google.colab.patches import cv2_imshow

Loading pretrained models

In [ ]:
# Load the pre-trained models
modeltf = InceptionV3(weights='imagenet', include_top=False)
modelpt = models.squeezenet1_0(pretrained=True)

Loading dataset

In [ ]:
import tensorflow as tf

batch_size = 1
img_size = (229, 229)
data_dir = "/content/drive/MyDrive/Dataset"

# Create the training dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
)

# Create the validation dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
)

class_names = dataset.class_names
print("Class names:", class_names)

Found 963 files belonging to 8 classes.
Using 771 files for training.
Found 963 files belonging to 8 classes.
Using 192 files for validation.
Class names: ['0', '135', '180', '225', '270', '315', '45', '90']


In [ ]:
len(test_dataset )

192

Phog

In [ ]:
def phog(image):
 gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 bins = 8  # Number of orientation bins
 levels = 3  # Number of pyramid levels
 hist = cv2.calcHist([gray], [0], None, [bins], [0, 256])
 hist /= np.sum(hist)  # Normalize the histogram
 return hist

LEP

In [ ]:
def lep_lbp(image, radius, points):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = []
    for y in range(radius, gray.shape[0] - radius):
        for x in range(radius, gray.shape[1] - radius):
            center = gray[y, x]
            energy = np.sum((gray[y-radius:y+radius+1, x-radius:x+radius+1] - center) ** 2)
            pattern = 0
            for i, (dx, dy) in enumerate(zip(np.cos(np.linspace(0, 2*np.pi, points)), np.sin(np.linspace(0, 2*np.pi, points)))):
                nx, ny = int(x + radius * dx), int(y + radius * dy)
                if gray[ny, nx] >= center:
                    pattern += 1 << i
            features.append((pattern, energy))

    return features.flatten()

Feature Extraction and Max Features,Low Features

In [ ]:
LBP_RADIUS = 1
LBP_POINTS = 8 * LBP_RADIUS
histRange = (0, 256)
histSize=256
accumulate = False
def LowLevelFeatures(image):
       phog =phog(image)
       b_hist = cv2.calcHist(bgr_planes, [0], None,[histSize], histRange, accumulate=accumulate)
       g_hist=cv2.calcHist(bgr_planes,[1],None,[histSize],histRange, accumulate=accumulate)
       r_hist = cv2.calcHist(bgr_planes, [2], None,[histSize], histRange, accumulate=accumulate)
       chf = np.concatenate((b_hist, g_hist, r_hist)).flatten()
       Lep=lep_lbp(image,2,4)
       return phog,chf,Lep

In [ ]:
googlenetfeatures= []
sequeezenetFeatures=[]
featuresCombined=[]
Y = []
maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)
for images, labels in dataset:
    threeD = np.squeeze(images, axis=0)
    phog,chf,lep= LowLevelFeatures(threeD)
    images = images / 255.0
    tf_features = modeltf.predict(images)
    img = np.transpose(images, (0, 3, 1, 2))
    pt_img = np.array(img)
    pt_img = torch.FloatTensor(pt_img)
    with torch.no_grad():
      pt_features = modelpt.features(pt_img).numpy()
    pooled_feature_map = maxpool(torch.from_numpy(tf_features))
    pooled_feature_map2=maxpool(torch.from_numpy(pt_features))
    phog=torch.from_numpy(phog)
    chf=torch.from_numpy(chf)
    Lep=torch.from_numpy(Lep)
    featuresCombined.append(np.concatenate([ np.array(pooled_feature_map ).flatten(), np.array(pooled_feature_map2).flatten(),
      np.array(phog).flatten(),np.array(chf).flatten().np.array(Lep).flatten()]))
    Y.append(labels)
Y = tf.concat(Y, axis=0)

In [ ]:
np.array(featuresCombined).shape

(771, 136704)

Test dataset

In [ ]:
testgooglenetfeatures= []
testsequeezenetFeatures=[]
featuresCombinedtest=[]
Ytest = []
maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)
for images, labels in val_dataset:
    threeD = np.squeeze(images, axis=0)
    phog,chf= LowLevelFeatures(threeD)
    images = images / 255.0
    tf_features = modeltf.predict(images)
    img = np.transpose(images, (0, 3, 1, 2))
    pt_img = np.array(img)
    pt_img = torch.FloatTensor(pt_img)
    with torch.no_grad():
      pt_features = modelpt.features(pt_img).numpy()
    pooled_feature_map = maxpool(torch.from_numpy(tf_features))
    pooled_feature_map2=maxpool(torch.from_numpy(pt_features))
    phog=torch.from_numpy(pt_features)
    chf=torch.from_numpy(chf)
    featuresCombinedtest.append(np.concatenate([ np.array(pooled_feature_map ).flatten(), np.array(pooled_feature_map2).flatten(),
      np.array(phog).flatten(),np.array(chf).flatten().flatten()]))
    Ytest.append(labels)
Ytest = tf.concat(Ytest, axis=0)

1/1 [==============================] - 0s 29ms/step


Deep fusion test dataset

Applying PCA

In [ ]:
pca = PCA(n_components=1)
featuresCombined = pca.fit_transform(featuresCombined)

Deep fusion training dataset


In [ ]:
np.array(featuresCombinedtest).shape

(192, 136704)

SVM multli classifier

In [ ]:
svm_model = SVC(kernel='rbf')
ovr_classifier = OneVsRestClassifier(svm_model)
ovr_classifier.fit(np.array(featuresCombined),Y)

OneVsRestClassifier(estimator=SVC())

Saving model

Accuracy

Accuracy with both high and low level features accuracy is low because of image quality and low and high features red
andancy

In [ ]:
accuracy =ovr_classifier.score(np.array(featuresCombinedtest),Ytest)
print("Accuracy:", accuracy)

Accuracy: 0.7760416666666666


Aveage Pooling Accuracy

In [ ]:
y_pred=ovr_classifier.predict(np.array(featuresCombinedtest))

Output Labels

Predicted

In [ ]:
y_pred

array([5, 6, 2, 5, 6, 7, 3, 6, 4, 2, 0, 4, 5, 7, 4, 5, 6, 0, 7, 7, 6, 5,
       5, 6, 6, 0, 0, 3, 7, 0, 7, 2, 6, 3, 5, 7, 0, 7, 5, 2, 2, 2, 6, 3,
       4, 3, 6, 0, 7, 7, 1, 6, 3, 0, 6, 0, 5, 2, 3, 2, 2, 3, 3, 5, 7, 3,
       4, 2, 5, 6, 3, 3, 1, 0, 2, 1, 2, 0, 2, 6, 2, 2, 1, 4, 5, 2, 5, 0,
       0, 4, 0, 7, 6, 2, 4, 6, 0, 7, 1, 1, 5, 4, 4, 3, 2, 6, 6, 6, 3, 1,
       2, 5, 2, 1, 6, 7, 7, 6, 4, 7, 2, 7, 2, 3, 6, 6, 6, 3, 7, 6, 5, 7,
       4, 1, 7, 7, 5, 2, 1, 7, 0, 1, 7, 4, 5, 7, 1, 7, 4, 4, 2, 2, 7, 7,
       1, 0, 4, 6, 1, 6, 2, 0, 1, 0, 5, 2, 4, 5, 7, 1, 2, 1, 1, 5, 1, 5,
       4, 7, 1, 0, 0, 2, 2, 0, 7, 6, 0, 0, 5, 7, 3, 4], dtype=int32)

Target

In [ ]:
Ytest

<tf.Tensor: shape=(192,), dtype=int32, numpy=
array([6, 6, 2, 4, 6, 7, 3, 4, 4, 5, 0, 4, 3, 7, 4, 5, 6, 0, 7, 7, 1, 5,
       3, 6, 6, 0, 3, 3, 4, 0, 3, 2, 1, 3, 5, 0, 0, 7, 5, 4, 2, 2, 6, 3,
       4, 3, 2, 0, 7, 7, 3, 6, 3, 0, 6, 0, 5, 0, 4, 2, 0, 3, 3, 6, 7, 3,
       4, 2, 5, 6, 3, 3, 1, 6, 2, 1, 0, 0, 2, 1, 2, 2, 1, 4, 5, 2, 5, 0,
       0, 4, 0, 3, 6, 2, 4, 4, 0, 3, 1, 1, 5, 4, 4, 3, 2, 6, 6, 2, 3, 1,
       2, 5, 2, 1, 6, 7, 3, 4, 4, 1, 1, 7, 2, 3, 7, 6, 6, 3, 7, 6, 6, 7,
       4, 3, 7, 7, 5, 2, 1, 7, 2, 1, 0, 4, 5, 3, 1, 7, 4, 4, 2, 2, 7, 7,
       3, 0, 4, 6, 1, 6, 2, 0, 3, 0, 5, 2, 4, 5, 7, 3, 0, 1, 1, 6, 1, 5,
       4, 0, 1, 0, 0, 0, 2, 0, 7, 6, 0, 0, 4, 3, 3, 4], dtype=int32)>

Accuracy when only high level features were considered

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score
accuracy = accuracy_score(Ytest, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9270833333333334


Prediction